In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits import mplot3d

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, Sequential, Input, Model
from tensorflow.keras.layers import *
from sklearn.cluster import KMeans

import math
import scipy
import pickle
import gc 

from IPython.display import display

In [2]:
tf.random.set_seed(0)

In [3]:
fires = pickle.load(open("/kaggle/input/ecuador-and-galapagos-wildfire-detection/fires.dat", "rb"))
nonfires = pickle.load(open("/kaggle/input/ecuador-and-galapagos-wildfire-detection/nonfires.dat", "rb"))

In [4]:
def return_idxs(data):
    idxs = np.argwhere(np.min(np.min(np.min(data, axis=1), axis=1), axis=1) != 0)
    idxs = idxs.reshape(idxs.shape[0])
    return idxs

In [5]:
idxs_fire = return_idxs(fires)
swir = fires[idxs_fire,:,:,5:7]

idxs_nonfire = return_idxs(nonfires)
swir_nonfire = nonfires[idxs_nonfire,:,:,5:7]

In [6]:
cirrus_x = np.concatenate((swir, swir_nonfire))

In [7]:
def cirrus_idxs(data):
    cirrus_band = 7
    max_range_cirrus = 500 # experimentally decided; for the images that actually have cirrus contamination, the range of the pixel values is much bigger
    lst = []
    for i in range(data.shape[0]):
        condition = (np.max(data[i,:,:,cirrus_band]) - np.min(data[i,:,:,cirrus_band])) >= max_range_cirrus
        if condition1:
            lst.append(i)
    return lst

In [8]:
cirrus_idxs_fires = cirrus_idxs(fires) # length: 2152
cirrus_idxs_nonfires = cirrus_idxs(nonfires) # length: 3168

In [9]:
def cluster_cirrus(data, cirrus_idxs_data):
    clustering = KMeans(3, init="random") # 3 clusters: Dense cloud, Scattered cloud, No Cloud
    clustered_imgs = []
    
    points = np.zeros((16384,3))
    for i in range(128):
        points[128*i:128*(i+1),0] = i
        points[128*i:128*(i+1),1] = np.arange(0,128,1)

    for ii in cirrus_idxs_data:
        pixels = []
        for i in range(128):
            for j in range(128):
                pixels.append(data[ii,:,:,cirrus_band][i][j])
        points[:,-1] = np.array(pixels)
        
        clustering.fit(points)
        clustered_img = clustering.labels_.reshape(128,128)
        clustered_imgs.append(clustered_img)
        
    clustered_imgs = np.array(clustered_imgs)
    
    return clustered_imgs

In [10]:
def confirm_clusters(clustered_data, cirrus_idxs_data):
    cirrus = np.zeros((clustered_imgs.shape[0], 128, 128))
    
    for i in range(len(cirrus_idxs_data)):
        coords_0 = np.argwhere(clustered_imgs[i]==0).T
        coords_1 = np.argwhere(clustered_imgs[i]==1).T
        coords_2 = np.argwhere(clustered_imgs[i]==2).T
        mean_0 = np.mean(data[cirrus_idxs_data[i],:,:,7][coords_0[0,:], coords_0[1,:]])
        mean_1 = np.mean(data[cirrus_idxs_data[i],:,:,7][coords_1[0,:], coords_1[1,:]])
        mean_2 = np.mean(data[cirrus_idxs_data[i],:,:,7][coords_2[0,:], coords_2[1,:]])
        sorted_means = sorted(list([mean_0, mean_1, mean_2]))
        
        if sorted_means[2] == mean_0:
            if sorted_means[1] == mean_1:
                dense = 0
                scattered = 1
                none = 2
            else:
                dense = 0
                scattered = 2
                none = 1
        elif sorted_means[2] == mean_1:
            if sorted_means[1] == mean_0:
                dense = 1
                scattered = 0
                none = 2
            else:
                dense = 1
                scattered = 2
                none = 0
        else:
            if sorted_means[1] == mean_0:
                dense = 2
                scattered = 0
                none = 1
            else:
                dense = 2
                scattered = 1
                none = 0

        cirrus[i][clustered_imgs[i]==none] = 0
        cirrus[i][clustered_imgs[i]==scattered] = 1
        cirrus[i][clustered_imgs[i]==dense] = 2
        
        return cirrus

In [11]:
clustered_fire = cluster_cirrus(fires)
clustered_nonfire = cluster_cirrus(nonfires)
clustered_fire = confirm_clusters(clustered_fire, cirrus_idxs_fire)
clustered_nonfire = confirm_clusters(clustered_nonfire, cirrus_idxs_nonfire)
cirrus = np.concatenate((clustered_fire, clustered_nonfire))

In [12]:
# Run this cell when training the model on segmented cirrus inputs
experimental = True
if experimental:
    cirrus_x = np.concatenate((cirrus_x, cirrus), axis=3)

In [13]:
# pickle.dump(cirrus_x, open("cirrus_x.dat", "wb"))

In [14]:
cirrus_y = np.concatenate((np.ones((len(cirrus_idxs_fires))), np.zeros((len(cirrus_idxs_nonfires)))))

In [15]:
X_t, X_v, y_t, y_v = train_test_split(cirrus_x, cirrus_y, test_size=0.2, stratify=cirrus_y, random_state=0)

In [16]:
X_t_mins = np.min(np.min(np.min(X_t, axis=0), axis=0), axis=0) 
X_t_maxs = np.max(np.max(np.max(X_t, axis=0), axis=0), axis=0) 
X_t = (X_t - X_t_mins) / (X_t_maxs - X_t_mins)
X_v = (X_v - X_t_mins) / (X_t_maxs - X_t_mins)

In [17]:
input_depth = 3 # Experimental model has 3 channels, Baseline model has 2 channels

def cirrus_model_builder():
    l2 = tf.keras.regularizers.l2(l=0.003)
    inputs = Input(shape=(128,128,input_depth))
    
    x = Conv2D(32, (7,7), padding="valid")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((3,3), strides=2)(x)
    
    x = Conv2D(64, (5,5), padding="valid")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)   
    x = MaxPooling2D((3,3), strides=2)(x)
  
    x = Conv2D(128, (5,5), padding="valid", kernel_regularizer=l2)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)   
    x = MaxPooling2D((3,3), strides=2)(x)
   
    x = Conv2D(256, (5,5), padding="valid", kernel_regularizer=l2)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((3,3), strides=2)(x)
    x = Dropout(0.3)(x)
    
    x = Flatten()(x)
    x = Dense(300)(x)
    x = Dropout(0.3)(x)
    x = Activation("relu")(x)

    x = Dense(150)(x)
    x = Dropout(0.2)(x)
    x = Activation("relu")(x)
    
    x = Dense(50)(x)
    x = Dropout(0.2)(x)
    x = Activation("relu")(x)
    
    x = Dense(16)(x)
    x = Dropout(0.1)(x)
    x = Activation("relu")(x)
    
    x = Dense(8)(x)
    x = Dropout(0.1)(x)
    x = Activation("relu")(x)
    x = Dense(1)(x)
    
    outputs = Activation("sigmoid")(x)
    model = Model(inputs, outputs)
    
    return model

In [18]:
cirrus_model = cirrus_model_builder()
cirrus_model.summary()

In [19]:
# tf.keras.utils.plot_model(cirrus_model, to_file='cirrus_model.png')

In [20]:
weight_pos_cirrus = cirrus_y.shape[0] / np.sum(cirrus_y) - 1

def loss_fn_cirrus(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    epsilon = 1e-4
    error = - (weight_pos_cirrus * y_true * tf.math.log(y_pred + epsilon) + (1 - y_true) * tf.math.log(1 - y_pred + epsilon))
    loss = tf.math.reduce_mean(error, axis=-1)
    
    return loss

In [21]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-4)
cirrus_model.compile(optimizer=adam, loss=loss_fn_cirrus, metrics=["binary_accuracy"])

In [22]:
history_cirrus = cirrus_model.fit(X_t, y_t, validation_data=(X_v, y_v), batch_size=32, epochs=200)

In [23]:
plt.plot(history_cirrus.history["loss"])
val_loss = history_cirrus.history["val_loss"][:9] + list(pd.Series(history_cirrus.history["val_loss"]).rolling(10).mean().dropna())
plt.plot(val_loss)
plt.show()

plt.plot(history_cirrus.history["binary_accuracy"])
val_metric = history_cirrus.history["val_binary_accuracy"][:9] + list(pd.Series(history_cirrus.history["val_binary_accuracy"]).rolling(10).mean().dropna())
plt.plot(val_metric)
plt.yticks([i for i in np.arange(0.5, 1.05, 0.05)])
plt.show()